<a href="https://colab.research.google.com/github/MahdiZaman/Spring2019-CAP5610/blob/master/hw1p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
###HW1 P1

import numpy as np
from keras.datasets import mnist
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()


Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [0]:
# Preparing the Train and Test Images (Adding a column of 1s for biases)
nfeatures = 28*28
nclasses = 1
ntrainsamples = train_images_original.shape[0] #60000
ntestsamples = test_images_original.shape[0]   #10000

#normalizing inputs
train_images = train_images_original.reshape((ntrainsamples, nfeatures))
train_images = train_images.astype('float32') / 255 
test_images = test_images_original.reshape((ntestsamples, nfeatures))
test_images = test_images.astype('float32') / 255

#adding a column of 1s for bias calculation
train_images_b = np.c_[np.ones((ntrainsamples, 1)), train_images]
test_images_b = np.c_[np.ones((ntestsamples,1)), test_images]

# # One hot encoding for train and test labels
# train_labels = np.eye(nclasses)[train_labels_original]
# test_labels = np.eye(nclasses)[test_labels_original]

In [0]:
def sigmoid(z):
    a = 1/(1+ np.exp(-z))
    return a

In [0]:
def sigderiv(x):
    return sigmoid(x)*(1-sigmoid(x))
    

In [0]:
def forwardprop(x, weight):
    xw = x.dot(weight)
    A = sigmoid(xw)
    return A

In [0]:
def mse_loss(y, y_hat):
    return 0.5*np.mean((y_hat-y)**2)

In [0]:
## Training the model

#initializing Weights and Bias(Bias added as a column of 1 on the left in weight array)

initial_weight = np.random.randn(nfeatures+1, nclasses)
print(initial_weight.shape)

weight = initial_weight
weight_path_mgd = []
weight_path_mgd.append(weight)

epochs = 20 #number of epochs
minibatchsize = 64 #number of samples in minibatch
lr = 0.001 #Learning Rate

for epoch in range(epochs):
    #shuffling the samples
    m = np.random.permutation(ntrainsamples)
    train_images_b = train_images_b[m]
    train_labels = train_labels[m]
    
    #minibatch SGD
    for i in range(0, ntrainsamples, minibatchsize):
        train_images_m = train_images_b[i : i+minibatchsize] 
        train_labels_m = train_labels[i : i+minibatchsize]
        #if train_images_m.shape[0] != 128: print(i+minibatchsize)
    
        A = forwardprop(train_images_m, weight)
        loss = cce_loss(train_labels_m, A)
        
        # Updating Weights and Biases
        dw = deriv(train_images_m, train_labels_m, A , minibatchsize)
        weight = weight - lr*dw
        weight_path_mgd.append(weight)
        
    print(('Epoch {}/{} --- Training Loss {}'.format(epoch+1 , epochs, np.mean(loss))))